In [1]:
import onnxruntime as ort
import onnx

In [2]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as T

import tensorflow.keras as keras
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

In [3]:
def run_onnx(model_path, **kwargs):
    kwargs = {k:np.array(v) for k,v in kwargs.items()}
    out = ort.InferenceSession(model_path)\
        .run(None,input_feed=kwargs)    
    return out

In [16]:
from pathlib import Path
import json
model_dir = Path('../../models/student-pruned-9h-900/9h-900-pruned-distilled')
model_params = json.load((model_dir/'config.json').open())
special_tokens = json.load((model_dir/'special_tokens_map.json').open())
tokenizer_config = json.load((model_dir/'tokenizer_config.json').open())

{'architectures': ['BertForSequenceClassification'],
 'attention_head_size': 26,
 'attention_probs_dropout_prob': 0.1,
 'cell': {},
 'emb_size': 312,
 'finetuning_task': 'boolq',
 'gradient_checkpointing': False,
 'hidden_act': 'relu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 312,
 'initializer_range': 0.02,
 'intermediate_size': 900,
 'layer_norm_eps': 1e-12,
 'max_position_embeddings': 512,
 'model_type': 'bert',
 'num_attention_heads': 9,
 'num_hidden_layers': 4,
 'pad_token_id': 0,
 'pre_trained': '',
 'structure': [],
 'type_vocab_size': 2,
 'vocab_size': 30522}

In [ ]:
# crop = T.RandomCrop(PT_IMG_SIZE[2:],pad_if_needed=True,padding_mode='reflect')
# resize = T.Resize(PT_IMG_SIZE[2:])
batch_dim = T.Lambda(lambda x:x[None,...])
to_tensor = T.ToTensor()

# content_tfm = T.Compose((resize, to_tensor, batch_dim))
# style_tfm = T.Compose((resize, to_tensor, batch_dim))

iters = 2
c_inp = torch.randn(1,3,512,512)#content_tfm(c)
s_inp = torch.randn(1,3,512,512)#style_tfm(s)

with torch.no_grad():
    c4_1 = c_inp.detach().clone()
    s4_1 = s_inp.detach().clone()
    for _ in range(iters):
        c4_1, c5_1 = run_onnx('encoder.onnx', image=c4_1)
        s4_1, s5_1 = run_onnx('encoder.onnx', image=s4_1)
        (c_tfm,) = run_onnx('transform.onnx', c4_1=c4_1,
                         s4_1=s4_1, c5_1=c5_1, s5_1=s5_1)
        (c_stylized,) = run_onnx('decoder.onnx', c_tfm=c_tfm)
        c4_1 = c_stylized#.detach().clone()
        s4_1 = s_inp#.detach().clone()
        c4_1 = c4_1.clip(0,1)